## Performing Real-Time Test

In [9]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import os
import matplotlib.pyplot as plt

In [10]:
# !jupyter nbconvert --to script Create_dataset.ipynb

In [11]:
from Extracting_Landmarks import extract_keypoints , mp_holistic , mp_drawing , mediapipe_detection , drawing_styled_landmarks
from Create_dataset import actions

In [12]:
# File Path to store the landmark data

DATA_PATH = os.path.join("LANDMARK_DATA")

# Defining the actions 
actions = np.array(['hello' , 'thankyou' , 'My' , 'sorry', 'Name' , 'You' , 'I am' , 'Nice' ,'Meet' , 'Fine', 'I'])
# Defining the number of videos per action

no_of_videos = 100

# Defining the number of frames per video

no_of_frames = 30

In [13]:
# actions = np.array(['hello' , 'thankyou' , 'iloveyou'])

Importing the model

In [14]:
with open('LRCN_model' , 'rb') as file:
 
 model = pickle.load(file)

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 1662, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 831, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 831, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 831, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 415, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 415, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 415, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 7, 26560)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │     3,403,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11)             │           363 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,282,691 (39.23 MB)

 Trainable params: 3,427,563 (13.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,855,128 (26.15 MB)

Displaying probability for words

In [16]:
colors = [
    (255, 0, 0),  # Red
    (0, 255, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0), # Yellow
    (255, 0, 255), # Magenta
    (0, 255, 255), # Cyan
    (128, 128, 128), # Gray
    (128, 0, 0),    # Maroon
    (0, 128, 0),    # Dark Green
    (0, 0, 128),    # Navy Blue
    (135, 206, 235) # Sky Blue
]


def prob_visualization(res , actions , input_frame , colors):
  
  output_frame = input_frame.copy()
  for num , prob in enumerate(res):
    cv2.rectangle(output_frame ,(0 , 60 + num*40) , (int(prob*100), 90+num*40) , colors[num] , -1)
    cv2.putText(output_frame , actions[num] , (0, 85 + num*40) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255 , 255 , 255) , 2 , cv2.LINE_AA)
  
  return output_frame

# def prob_visualization(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         # Ensure color assignment even if `colors` has fewer entries than `res`
#         color = colors[num % len(colors)]
#         cv2.rectangle(output_frame, 
#                       (0, 60 + num * 40), 
#                       (int(prob * 100), 90 + num * 40), 
#                       color, 
#                       -1)
#         cv2.putText(output_frame, 
#                     actions[num], 
#                     (0, 85 + num * 40), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 
#                     1, 
#                     (255, 255, 255), 
#                     2, 
#                     cv2.LINE_AA)
#     return output_frame


In [18]:
# Checking video camera availablity 

sequence = []
sentence = []
threshold = 0.9
res = None
predictions = []
 
cap = cv2.VideoCapture(0)  # accesses the webcam

with mp_holistic.Holistic(min_detection_confidence= 0.7 , min_tracking_confidence= 0.7) as holistic:
   while cap.isOpened():  #This loop continiously read frames from the camera cap

      # Reading the frames
      ret , frame = cap.read() # ret is a boolean variable that is True is a frame is being read

      # Using Detection Function

      image , landmark_results = mediapipe_detection(frame , holistic )
      print(landmark_results)

      #Drawing Landmarks onto the frames on the screen
      drawing_styled_landmarks(image , landmark_results)

      # Adding prediction logic
      keypoints = extract_keypoints(landmark_results)
      sequence.append(keypoints)
      sequence = sequence[-30:]
   

      if len(sequence) == 30:
         res = model.predict(np.expand_dims(sequence , axis = 0 ))[0]
         predictions.append(np.argmax(res))

      # Visualization logic
      
      if len(predictions) >= 10 and np.unique(predictions[-10:])[0] == np.argmax(res):
         if res is not None and res[np.argmax(res)] > threshold:

            if len(sentence) > 0:

               if actions[np.argmax(res)] != sentence[-1]:
                  sentence.append(actions[np.argmax(res)])
            
            else: 

               sentence.append(actions[np.argmax(res)])
      
      if len(sentence) > 3:
         sentence = sentence[-3:]
      
      #Prob Viz
      if res is not None:
       image = prob_visualization(res , actions , image , colors)

      cv2.rectangle(image, (0, 0), (image.shape[1], 40), (245, 117, 16), -1)
      cv2.putText(image , ' '.join(sentence) , (3,30) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (255 , 255 , 255) , 2 , cv2.LINE_AA)

      # Displaying frames to screen 
      cv2.imshow('OpenCV' , image) 

      if cv2.waitKey(10) & 0xFF == ord('q'):
         break
      
   cap.release()
   cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti